# Task 3
## Creating alternative differential expression modules
---
## Relevance:  
While tools like sceptre are very accurated calculating differential perturbation it can take a long time to calculate the differential expression for all genes. 
Another researchers can have interest in extend the differential expression analysis to tools. The aim of this task is to create a module that can be used to calculate differential expression using a mudata contaning  a mudata object with the informations about guides and scRNA-seq , covariates and coordinate information 

## Practical description: TODO

-  [x] Download the mudata_results.h4mu and transform in a mudata_processed_guides_and_scrna.h5mu by removing unecessary information (code already provided "Downloading the File and creating mudata with guides and scRNA")
-  [ ] Create the enviroment by running the provided code. Check on  "Creating the enviroment" in the last notebook session
-  [ ] Create a nextflow process called differential_pertubation that takes as input the mudata_processed_guides_and_scrna.h5mu and a parameter contaning the information about in_trans=TRUE or FALSE.
-  [ ] Use the script  differential_pertubation.py to calculate the differential expression.
- [ ] return all the result in a dir called  results_dir
- [ ] Consider if we need a single file for each diff perturbation tool or if we can receive the current mudata and  

# Downloading and preprocessing experiment data

## Load MuData containing scRNA data and test results

In [1]:
!pwd

/home/jovyan


In [2]:
!wget https://www.dropbox.com/s/5jysq76bttem3jd/mudata_results.h5mu?dl=1 -O mudata_results.h5mu

--2023-04-03 15:28:35--  https://www.dropbox.com/s/5jysq76bttem3jd/mudata_results.h5mu?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/5jysq76bttem3jd/mudata_results.h5mu [following]
--2023-04-03 15:28:36--  https://www.dropbox.com/s/dl/5jysq76bttem3jd/mudata_results.h5mu
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucec8a7756382efe1128dedc366f.dl.dropboxusercontent.com/cd/0/get/B5dB2U89Hr1-yybV1aw4kpfx6ljmnjHBQANXLOePiAikKe9wr1OK7uOl5qizlvuF9t5tMkM34kc1mJofN8nPck-cnWB_Iji1zlhuN_PZsSp0UdlRk21zCb-7xROit4uQgNeGNIaEN8WeBJp32bxeHerUWt_HIrHWt2212roktcbHi22Qq_OvVmvC0H_REg3v9D0/file?dl=1# [following]
--2023-04-03 15:28:37--  https://ucec8a7756382efe1128dedc366f.dl.dropboxusercontent.com/cd/0/get/B5dB2U89Hr1-yybV1aw4kpfx6ljmnjHBQANX

### Inspect combined MuData object containing both raw data and test summary metrics
This is the format for data & test results that is currently expected for use in task 4, so our output should look similar to this 

In [3]:
import mudata as md
mu_results = md.read('/home/jovyan/mudata_results.h5mu')
mu_results

/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/opt/conda/lib/python3.10/site-packages/mudata/_core/mudata.py:458: UserWarning: Cannot join columns with the same name because var_names are intersecting.
  warnings.warn(


MuData object with n_obs × n_vars = 5218 × 17212
  4 modalities
    guides:	5109 x 98
      obs:	'bath_number', 'percent_mito', 'log_number_of_detected_genes', 'log_total_gene_count', 'log_total_guide_count'
      var:	'feature_name', 'guide_chr', 'guide_end', 'guide_start', 'guide_number', 'target_elements'
    scRNA:	5109 x 16030
      obs:	'bath_number', 'percent_mito', 'log_number_of_detected_genes', 'log_total_gene_count', 'log_total_guide_count'
      var:	'feature_name', 'n_cells', 'transcript_chr', 'transcript_start', 'transcript_end', 'ensg'
    result_guides:	68 x 542
      obs:	'feature_name', 'guide_chr', 'guide_end', 'guide_start', 'guide_number', 'target_elements'
      var:	'feature_name', 'n_cells', 'transcript_chr', 'transcript_start', 'transcript_end', 'ensg'
      layers:	'adj_pvalue', 'log_fold_change', 'significant', 'z_value'
    result_elements:	41 x 542
      obs:	'element', 'element_chr', 'element_start', 'element_end'
      var:	'feature_name', 'n_cells', 'transcript_chr', 'transcript_start', 'transcript_end', 'ensg'
      layers:	'sig_not_adj'

### Remove test results from MuData (guides and genes only) and inspect

In [4]:
mu_guides_and_rna = md.MuData({'guides': mu_results['guides'], 'scRNA':mu_results['scRNA']})
mu_guides_and_rna.write('mudata_processed_guides_and_scrna.h5mu')

In [5]:
mu_guides_and_rna['guides']

AnnData object with n_obs × n_vars = 5109 × 98
    obs: 'bath_number', 'percent_mito', 'log_number_of_detected_genes', 'log_total_gene_count', 'log_total_guide_count'
    var: 'feature_name', 'guide_chr', 'guide_end', 'guide_start', 'guide_number', 'target_elements'

In [6]:
mu_guides_and_rna['guides'].obs

,bath_number,percent_mito,log_number_of_detected_genes,log_total_gene_count,log_total_guide_count
AAACCTGAGAGGGCTT,1,0.000000,5.513429,5.517453,0.000000
AAACCTGCAAATACAG,1,0.022816,8.161660,8.161946,0.000000
AAACCTGCACTCGACG,1,0.051611,7.784473,7.784889,0.000000
AAACCTGCAGTCTTCC,1,0.053218,7.934155,7.934513,0.693147
AAACCTGCATGCCTAA,1,0.045982,8.531490,8.531688,0.000000
...,...,...,...,...,...
TTTGGTTTCCTGCAGG,1,0.027573,7.791110,7.791523,2.079442
TTTGGTTTCGTCCGTT,1,0.027464,8.423102,8.423322,0.693147
TTTGTCAAGCGAGAAA,1,0.040875,7.649693,7.650169,0.000000
TTTGTCACACATAACC,1,0.031246,8.506739,8.506941,1.386294


In [7]:
mu_guides_and_rna['guides'].var

,feature_name,guide_chr,guide_end,guide_start,guide_number,target_elements
feature_id,,,,,,
BROX|1_sgrna_chr1:222712600:222712620,BROX|1_sgrna_chr1:222712600:222712620,chr1,222712620,222712600,1,BROX
BROX|2_sgrna_chr1:222712744:222712764,BROX|2_sgrna_chr1:222712744:222712764,chr1,222712764,222712744,2,BROX
CCND2|1_sgrna_chr12:4273763:4273783,CCND2|1_sgrna_chr12:4273763:4273783,chr12,4273783,4273763,1,CCND2
CCND2|2_sgrna_chr12:4273832:4273852,CCND2|2_sgrna_chr12:4273832:4273852,chr12,4273852,4273832,2,CCND2
CEBPB|1_sgrna_chr20:50190830:50190850,CEBPB|1_sgrna_chr20:50190830:50190850,chr20,50190850,50190830,1,CEBPB
...,...,...,...,...,...,...
UAP1|2_sgrna_chr1:162561996:162562016,UAP1|2_sgrna_chr1:162561996:162562016,chr1,162562016,162561996,2,UAP1
UBE2T|1_sgrna_chr1:202341918:202341938,UBE2T|1_sgrna_chr1:202341918:202341938,chr1,202341938,202341918,1,UBE2T
UBE2T|2_sgrna_chr1:202341907:202341927,UBE2T|2_sgrna_chr1:202341907:202341927,chr1,202341927,202341907,2,UBE2T


In [8]:
mu_guides_and_rna['scRNA']

AnnData object with n_obs × n_vars = 5109 × 16030
    obs: 'bath_number', 'percent_mito', 'log_number_of_detected_genes', 'log_total_gene_count', 'log_total_guide_count'
    var: 'feature_name', 'n_cells', 'transcript_chr', 'transcript_start', 'transcript_end', 'ensg'

In [9]:
mu_guides_and_rna['scRNA'].var

,feature_name,n_cells,transcript_chr,transcript_start,transcript_end,ensg
feature_name,,,,,,
AL627309.1,AL627309.1,79,NOT_FOUND,0,0,ENSG00000238009
AL627309.5,AL627309.5,417,NOT_FOUND,0,0,ENSG00000241860
AP006222.1,AP006222.1,240,NOT_FOUND,0,0,ENSG00000228463
AP006222.2,AP006222.2,51,NOT_FOUND,0,0,ENSG00000286448
AL732372.2,AL732372.2,90,NOT_FOUND,0,0,ENSG00000237094
...,...,...,...,...,...,...
AL592183.1,AL592183.1,250,NOT_FOUND,0,0,ENSG00000273748
AC007325.4,AC007325.4,58,NOT_FOUND,0,0,ENSG00000278817
AL354822.1,AL354822.1,525,NOT_FOUND,0,0,ENSG00000278384


In [10]:
mu_guides_and_rna['scRNA'].obs

,bath_number,percent_mito,log_number_of_detected_genes,log_total_gene_count,log_total_guide_count
AAACCTGAGAGGGCTT,1,0.000000,5.513429,5.517453,0.000000
AAACCTGCAAATACAG,1,0.022816,8.161660,8.161946,0.000000
AAACCTGCACTCGACG,1,0.051611,7.784473,7.784889,0.000000
AAACCTGCAGTCTTCC,1,0.053218,7.934155,7.934513,0.693147
AAACCTGCATGCCTAA,1,0.045982,8.531490,8.531688,0.000000
...,...,...,...,...,...
TTTGGTTTCCTGCAGG,1,0.027573,7.791110,7.791523,2.079442
TTTGGTTTCGTCCGTT,1,0.027464,8.423102,8.423322,0.693147
TTTGTCAAGCGAGAAA,1,0.040875,7.649693,7.650169,0.000000
TTTGTCACACATAACC,1,0.031246,8.506739,8.506941,1.386294


# Creating a conda enviroment
We're going to specify a conda environment to run our analysis, here's an example environment that uses scanpy

In [11]:
%%writefile perturb.yaml
name: perturb-conda-jamboree
channels:
    - conda-forge
    - defaults
dependencies:
    - anndata
    - scanpy
    - mudata
    - pandas >= 1.5.3
    - session-info
    - pip

Overwriting perturb.yaml


In [12]:
!mamba env create -f perturb.yaml --force

conda-forge/linux-64                                        Using cache
conda-forge/noarch                                          Using cache
[+] 0.0s
[+] 0.1s
pkgs/main/linux-64 ━━━━━━━━━╸━━━━━━━━━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.1s
pkgs/main/noarch   ━━━╸━━━━━━━━━━━━━━━╸━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.1s
pkgs/r/linux-64    ━━╸━━━━━━━━━━━━━━━╸━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.1s
pkgs/r/noarch      ━╸━━━━━━━━━━━━━━━╸━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.1spkgs/main/linux-64                                            No change
pkgs/main/noarch                                              No change
pkgs/r/noarch                                                 No change
pkgs/r/linux-64                                               No change


Looking for: ['anndata', 'scanpy', 'mudata', "pandas[version='>=1.5.3']", 'session-info', 'pip']


Transaction

  Prefix: /home/jovyan/.conda/envs/perturb-conda-jamboree

  Updating specs:

   - anndata
   - scanpy
   - mudat

In [13]:
!conda env list

# conda environments:
#
perturb-conda-jamboree     /home/jovyan/.conda/envs/perturb-conda-jamboree
perturbvi-conda-jamboree     /home/jovyan/.conda/envs/perturbvi-conda-jamboree
base                     /opt/conda
pygenomictracks          /opt/conda/envs/pygenomictracks



In [14]:
!source activate perturb-conda-jamboree

# Developing a nextflow pipeline module

## Nextflow job specification

In [15]:
%%writefile differential_pertubation.nf


params.IN_TRANS = 'TRUE'
params.MU_GUIDE_AND_SCRNA = '/home/jovyan/mudata_processed_guides_and_scrna.h5mu'



useMamba = true
//place to cash conda/mamba packages
cacheDir =  '/home/jovyan/' 

process differential_pertubation {

   debug true
   conda '/home/jovyan/.conda/envs/perturb-conda-jamboree'
    
   input:
   path (mudata_guides_rna)
   val (in_trans)
   output:
   path 'results_dir', emit: differential_results_dir
   script:
    """
     mkdir -p results_dir
     python /home/jovyan/differential_pertubation.py ${mudata_guides_rna} ${in_trans}
    """
    
}

workflow {
    differential_pertubation(params.MU_GUIDE_AND_SCRNA,
                             params.IN_TRANS )
    
}


Overwriting differential_pertubation.nf


## Python script for nextflow to run
(You can also substitute in a R script or a wrapper around another tool, just change the extension and update the .nf file above)

In [16]:
%%writefile differential_pertubation.py
import sys
import os
import mudata as md


mu_guides_and_rna = md.read(sys.argv[1])
in_trans = sys.argv[2]

### do some analysis
# ...

#save all the results (or a mudata witht the results) in the results_dir dir. 

output_file = 'results_dir/'+os.path.basename(sys.argv[1]).replace('.h5mu','-final.h5mu')
mu_guides_and_rna.write_h5mu(output_file)


Overwriting differential_pertubation.py


In [17]:
!nextflow run differential_pertubation.nf -w diff_perturb_test -with-conda

N E X T F L O W  ~  version 22.10.6
Launching `differential_pertubation.nf` [maniac_pesquet] DSL2 - revision: 64f37edfb1
[-        ] process > differential_pertubation -

executor >  local (1)
[76/e73dd8] process > differential_pertubation [  0%] 0 of 1

executor >  local (1)
[76/e73dd8] process > differential_pertubation [100%] 1 of 1 ✔




Your output data will show up in a subfolder of `diff_perturb_test`

In [18]:
ls -lt diff_perturb_test

total 52
drwxr-sr-x 3 jovyan users 4096 Apr  3 15:29 76/
drwxr-sr-x 3 jovyan users 4096 Apr  3 15:20 1b/
drwxr-sr-x 4 jovyan users 4096 Apr  3 15:16 39/
drwxr-sr-x 3 jovyan users 4096 Apr  3 15:14 3d/
drwxr-sr-x 3 jovyan users 4096 Apr  3 15:13 31/
drwxr-sr-x 3 jovyan users 4096 Apr  3 15:10 10/
drwxr-sr-x 3 jovyan users 4096 Apr  3 15:08 e9/
drwxr-sr-x 3 jovyan users 4096 Apr  3 14:53 37/
drwxr-sr-x 3 jovyan users 4096 Apr  3 14:51 d7/
drwxr-sr-x 3 jovyan users 4096 Apr  3 14:45 21/
drwxr-sr-x 3 jovyan users 4096 Apr  3 14:43 d5/
drwxr-sr-x 3 jovyan users 4096 Apr  3 14:17 f8/
drwxr-sr-x 3 jovyan users 4096 Apr  3 14:17 e2/
